# 허깅페이스 사전학습 모델 사용

소규모의 데이터셋과 모델에 대해서는 일반사용자도 학습부터 추론의 처음부터 끝까지 수행하는 것이 가능합니다.

최근에는, 대규모 데이터셋의 접근이 용이해지고 GPT와 같은 대규모 언어 모델 (LLM)의 성공으로

기대하는 성능을 얻기 위해 사용하는 학습 데이터의 규모와 모델 파라미터의 크기가 점차 커져갔습니다.

그러나, 계산 자원이 부족한 대부분의 일반 사용자는 대규모의 데이터셋과 대규모 모델을 처리한 후 학습시키는 것 조차 불가능합니다.

한 가지 대안으로는, 대규모의 데이터셋으로 사전학습된 모델 가중치만을 다운로드 받아

추론만 하는데 사용하거나 소규모의 데이터셋으로 다시 학습할 수도 있습니다.

이 예제 코드에서는 huggingface의 transformers 패키지를 사용하여 사전학습 모델을 다운로드 받고 활용하는 방법을 간략하게 소개합니다.


# 패키지 설치

- transformers
- datasets
- evaluate
- accelerate
- sentencepiece

In [ ]:
%pip install transformers datasets evaluate accelerate sentencepiece

Note: you may need to restart the kernel to use updated packages.


# 모델 불러오기

자연어처리에서 huggingface에서 모델을 다운로드받아 활용하는 두가지 방법이 있습니다.

1. 모델 중점: Model (과 Tokenizer) 사용
2. 태스크 중점: pipeline 사용

## Model 사용

huggingface의 Model 클래스에 맞추어서 가중치를 로드합니다.

일반적인 경우 사용하고자 하는 모델에 맞는 적절한 클래스를 찾아 초기화해주어야 합니다.

다음은 일반목적 언어모델인 flan-t5를 다운로드하고 사용해보는 예제입니다.

**Note:** huggingface는 `AutoModel` (과 `AutoTokenizer`)라는 자동으로 적절한 클래스로 초기화 시켜주는 도우미 클래스도 제공합니다.

In [1]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

model = None
tokenizer = None
pipe = None

def release():
    """gpu 메모리를 비우기 위한 함수입니다."""
    global model, tokenizer, pipe
    del model, tokenizer, pipe
    model = None
    tokenizer = None
    pipe = None
    torch.cuda.empty_cache()

In [2]:
from transformers import AutoTokenizer, T5ForConditionalGeneration

model_id = "google/flan-t5-large"

model = T5ForConditionalGeneration.from_pretrained(model_id, device_map=device)
tokenizer = AutoTokenizer.from_pretrained(model_id, device_map=device)

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

c:\Users\xaiseung24\.conda\envs\general\lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\xaiseung24\.cache\huggingface\hub\models--google--flan-t5-large. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [4]:
text = "How does the gravity work?"
input_tensor = tokenizer.encode(text, return_tensors="pt").to(device)
output_tensor = model.generate(input_tensor, do_sample=False, max_length=32)
print(tokenizer.decode(output_tensor[0], skip_special_tokens=True))

Gravity is the force that pulls objects together.


## 파이프라인 사용

transformers 패키지에는 `pipeline()`이라는 추상적인 메소드로 사전에 정의된 파이프라인을 불러올 수 있습니다.

`pipeline()`에 텍스트 생성, 텍스트 분류, 추출적 질의응답, 번역 등 태스크를 지정하고 이를 지원하는 모델 ID를 입력으로 주면, 이에 맞는 `Pipeline` 클래스를 반환해줍니다.

다음부터는 파이프라인으로 `flan-t5-large`를 활용하는 예시 코드들입니다. 위의 코드에서 했던 tokenizer의 전/후처리를 pipeline 내부에서 자동으로 수행해줍니다.

In [3]:
from transformers import pipeline

release()

model_id = "google/flan-t5-large"

pipe = pipeline("text2text-generation", model=model_id, max_new_tokens=128, device_map=device)

print(pipe("How does the gravity work?"))

[{'generated_text': 'Gravity is the force that pulls objects together.'}]


# 추출적 질의응답 (Extractive Question-Answering)

추출적 질의응답은 맥락과 맥락에 대한 질문이 주어졌을 때, 질문에 대한 대답을 맥락으로부터 추출하여 대답하는 태스크를 말합니다.

입력으로 context와 question이 주어지며, 모델은 question에 대하여 대답이 될 수 있는 context의 일부분을 반환해야 합니다.

모델로 허깅페이스허브에 있는 한국어 추출적 QA 모델 `jihoonkimharu/bert-base-klue-mrc-finetuned`을 사용할 것입니다.

pipeline task 이름으로는 `question-answering`을 사용합니다.

In [4]:
from transformers import pipeline

release()

model_id = "jihoonkimharu/bert-base-klue-mrc-finetuned"

pipe = pipeline("question-answering", model=model_id, device_map=device)

body_text = """사과나무의 원산지는 발칸반도로 알려져 있으며 B.C. 20세기 경의 스위스 토굴 주거지에서 탄화된 사과가 발굴된 것으로 보아 서양사과는 4,000년 이상의 재배 역사를 가진 것으로 추정된다.
그리스 시대에는 재배종, 야생종을 구분한 기록이 있고 접목 번식법이 이미 소개 되어 있을 정도로 재배 기술이 진보되었다.
로마시대에는 Malus 또는 Malum이란 명칭으로 재배가 성향하였고 그 후 16-17세기에 걸쳐 유럽 각지에 전파되었다.
17세기에는 미국에 전파되었고 20세기에는 칠레 등 남미 각국에 전파되었다.
"""


print(pipe({"context": body_text, "question": "미국에 사과가 전파된 시기는 언제인가?"}))

print(pipe({"context": body_text, "question": "서양 사과가 역사는 어느정도의 시간인가?"}))

config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

c:\Users\xaiseung24\.conda\envs\general\lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\xaiseung24\.cache\huggingface\hub\models--jihoonkimharu--bert-base-klue-mrc-finetuned. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/499 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

c:\Users\xaiseung24\.conda\envs\general\lib\site-packages\transformers\pipelines\question_answering.py:391: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(


{'score': 0.8773866891860962, 'start': 244, 'end': 250, 'answer': '17세기에는'}
{'score': 0.8648744821548462, 'start': 76, 'end': 82, 'answer': '4,000년'}


# 감정 분석 (Sentiment analysis)

감정 분석은 입력된 텍스트에 대한 긍,부정 또는 분노, 행복, 공포 등의 어떤 감정이 포함되어 있는지 판단하는 태스크입니다.

BERT 기반 영어 감정 분석 모델 `finiteautomata/bertweet-base-sentiment-analysis`을 활용해보는 예시입니다.

해당 모델은 입력 텍스트에 대한 긍정 (POS), 부정 (NEG), 중립 (NEU)으로 감정 분석을 하여 예측한 클래스를 출력합니다.

pipeline task 이름으로는 `text-classification`을 사용합니다.

In [5]:
from transformers import pipeline

release()

model_id = "finiteautomata/bertweet-base-sentiment-analysis"

pipe = pipeline("text-classification", model=model_id, device_map=device)

print(pipe("Never gonna let you down.")) # 절때 실망시키지 않을께.
print(pipe("Shut up, I don't wanna hear you.")) # 닥쳐, 너한테 듣고싶지 않아.
print(pipe("What time is it?")) # 몇시야?

config.json:   0%|          | 0.00/949 [00:00<?, ?B/s]

c:\Users\xaiseung24\.conda\envs\general\lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\xaiseung24\.cache\huggingface\hub\models--finiteautomata--bertweet-base-sentiment-analysis. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


pytorch_model.bin:   0%|          | 0.00/540M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/338 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/843k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/22.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0


[{'label': 'POS', 'score': 0.9094418287277222}]
[{'label': 'NEG', 'score': 0.9702497720718384}]
[{'label': 'NEU', 'score': 0.9634673595428467}]


# 마스크 채우기 (Fill Mask)

마스크 채우기는 주어진 텍스트에서 `[MASK]` 로 비어있는 부분에 들어갈 단어를 예측합니다.

이 태스크는 주로 사전학습 단계에서 다루어 자연어의 이해 자체를 학습하는데 사용합니다.

다음은 유명한 사전학습 자연어 모델 `BERT`를 사용해보는 예시입니다.

pipeline task 이름으로 `fiil-mask`을 사용합니다.

In [6]:
from transformers import pipeline

release()

model_id = "google-bert/bert-large-uncased"

pipe = pipeline("fill-mask", model=model_id, device_map=device)

print(pipe("apple's color is [MASK].", top_k=1))

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

c:\Users\xaiseung24\.conda\envs\general\lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\xaiseung24\.cache\huggingface\hub\models--google-bert--bert-large-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of the model checkpoint at google-bert/bert-large-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dens

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

[{'score': 0.1278262436389923, 'token': 2417, 'token_str': 'red', 'sequence': "apple ' s color is red."}]


# 사전학습 모델 파인튜닝하기

우리가 원하는 태스크에 대해서 인공지능을 적용하고자 할 때, 강력한 사전학습 파라미터에서 시작한다면 처음부터 학습하는 경우에 비해 더 좋은 성능을 얻으면서 시간을 아낄 수 있습니다.

왜냐하면, 대규모 말뭉치를 통해서 사전학습된 모델은 그 언어에 대해서 풍부히 이해하고 있기 때문에 구체적인 태스크에 대해 새롭게 훈련을 해줬을 때 이 이해를 바탕으로 좋은 성능을 빠르게 얻어내기 때문입니다.

이렇게 대규모 말뭉치에 사전학습 후 파인튜닝에 사용되는 대규모 언어모델을 **Foundation Model**이라 부릅니다.

다음을 영화 긍부정 리뷰에 대하여 사전학습된 모델을 파인튜닝하는 간단한 예제입니다.

*Note: 빠른 테스트를 위해 데이터셋의 일부만 사용했기에 기대하는 성능보다 조금 저조합니다.*

In [54]:
from transformers import pipeline
import numpy as np
import torch
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    )
from datasets import load_dataset
import evaluate

release()


model_id = "distilbert-base-cased"

model = AutoModelForSequenceClassification.from_pretrained(model_id, device_map = device, )#torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained(model_id)

raw_dataset = load_dataset("stanfordnlp/imdb")
del raw_dataset["unsupervised"]

# 평가 지표 정의
metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions,references=labels)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [34]:
# 데이터셋 예시
raw_dataset["train"][0]

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [55]:
# 데이터셋의 일부만 사용
small_train_dataset = raw_dataset["train"].shuffle(seed=42).select(range(2000))
small_eval_dataset = raw_dataset["test"].shuffle(seed=42).select(range(400))

# 전처리
max_length = 48
def tokenize_fn(examples):
    return tokenizer(examples["text"], max_length=max_length, padding="max_length", truncation=True)

small_train_dataset = small_train_dataset.map(tokenize_fn, batched=True)
small_eval_dataset = small_eval_dataset.map(tokenize_fn, batched=True)



Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [57]:
# 학습 준비 - Trainer 변수

trainer_args = TrainingArguments(
    output_dir="distilbert_imdb",
    fp16=True,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=1,
    optim="adafactor",
    eval_strategy="epoch",
    save_strategy="no",
    report_to="none")

trainer = Trainer(
    model=model,
    processing_class=tokenizer,
    args=trainer_args,
    train_dataset = small_train_dataset,
    eval_dataset = small_eval_dataset,
    compute_metrics=compute_metrics,
)


In [58]:
# 최초 성능 체크.
trainer.evaluate()
# 평가가 안되고 오류가 발생하면 
# per_device_train_batch_size와 per_device_eval_batch_size의 값을 줄인다.

  0%|          | 0/7 [00:00<?, ?it/s]

{'eval_loss': 0.6964366436004639,
 'eval_model_preparation_time': 0.0019,
 'eval_accuracy': 0.51,
 'eval_runtime': 9.558,
 'eval_samples_per_second': 41.85,
 'eval_steps_per_second': 0.732}

정확도가 0.5에 가깝게 나온다면, 거의 무작위로 예측한다는 뜻입니다.

우리가 불러온 사전학습된 BERT 모델은 영화의 긍부정 리뷰를 위해 훈련된 것이 아니기 때문입니다.

이제 파인튜닝을 통해 영화 긍부정에 대해서 훈련하며 정확도를 높일 수 있습니다.

In [59]:
# 학습
trainer.train()

  0%|          | 0/32 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

{'eval_loss': 0.5197383761405945, 'eval_model_preparation_time': 0.0019, 'eval_accuracy': 0.7675, 'eval_runtime': 10.936, 'eval_samples_per_second': 36.577, 'eval_steps_per_second': 0.64, 'epoch': 1.0}
{'train_runtime': 227.4126, 'train_samples_per_second': 8.795, 'train_steps_per_second': 0.141, 'train_loss': 0.6244255304336548, 'epoch': 1.0}


TrainOutput(global_step=32, training_loss=0.6244255304336548, metrics={'train_runtime': 227.4126, 'train_samples_per_second': 8.795, 'train_steps_per_second': 0.141, 'total_flos': 24837637248000.0, 'train_loss': 0.6244255304336548, 'epoch': 1.0})

In [60]:
# 학습 후 평가
trainer.evaluate()

  0%|          | 0/7 [00:00<?, ?it/s]

{'eval_loss': 0.5197383761405945,
 'eval_model_preparation_time': 0.0019,
 'eval_accuracy': 0.7675,
 'eval_runtime': 10.7883,
 'eval_samples_per_second': 37.077,
 'eval_steps_per_second': 0.649,
 'epoch': 1.0}

마지막에는 정확도가 75% 정도 나타납니다.

모델을 저장해봅시다.

In [ ]:
trainer.save_model("./distilbert_imdb/latest",)

# 학습된 모델 불러오기

In [ ]:
from transformers import pipeline
import numpy as np
import torch
from transformers import (
    AutoModelForSequenceClassification,
    DistilBertForMaskedLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    )
from datasets import load_dataset
import evaluate

release()

# 저장된 모델 디렉토리
model_path = "./distilbert_imdb/latest"

# 저장된 모델 불러오기 (model_id 대신 디렉토리를 넣기)
model = AutoModelForSequenceClassification.from_pretrained(model_path, device_map = device, )#torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [ ]:
# trainer 재정의
trainer_args = TrainingArguments(
    output_dir="distilbert_imdb",
    fp16=True,
    per_device_eval_batch_size=64,
    num_train_epochs=1,
    eval_strategy="epoch",
    save_strategy="no",
    report_to="none")

trainer = Trainer(
    model=model,
    processing_class=tokenizer,
    args=trainer_args,
    train_dataset = small_train_dataset,
    eval_dataset = small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
# 평가
trainer.evaluate()

  0%|          | 0/7 [00:00<?, ?it/s]

{'eval_loss': 0.5197383761405945,
 'eval_model_preparation_time': 0.0016,
 'eval_accuracy': 0.7675,
 'eval_runtime': 11.6996,
 'eval_samples_per_second': 34.189,
 'eval_steps_per_second': 0.598}